In [ ]:
pip install dmba==0.0.18

     |████████████████████████████████| 11.8 MB 11.8 MB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from dmba import regressionSummary

no display found. Using non-interactive Agg backend


# Linear Regression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sido = pd.read_csv("/content/drive/MyDrive/2022_IPS/final_data/지역명_사망자수(train).csv")
sido = sido.drop(columns=['Unnamed: 0'])
sido.head()

,지역명,병원수,격리병상수,남은 격리병상수,총인구,요양기관수,60세 이상 - 계,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상,확진자수,사망자수
0,강원도,18.0,539.0,75.0,1347375.0,34.0,369970,116558,81374,56864,53119,38211,23844,440985.0,746
1,경상남도,35.0,940.0,176.0,3295942.0,129.0,774696,253193,179622,128375,99667,70217,43622,1052389.0,1134
2,경상북도,29.0,1046.0,271.0,2583157.0,111.0,719154,213011,162533,119640,100383,75400,48187,731978.0,1203
3,광주광역시,19.0,240.0,55.0,1477573.0,60.0,290547,96662,65287,52134,38610,23826,14028,518968.0,520
4,대구광역시,18.0,876.0,249.0,2410700.0,75.0,559877,182705,133421,95965,73211,47435,27140,734292.0,1243


In [ ]:
columns = ['loc','cnt_hos','bed','remain_bed','pop','nurse','total60','60-64','65-69','70-74','75-79','80-84','85-','conf','death']
var_name = pd.DataFrame(sido.columns, columns, columns=['meaning'])
var_name

,meaning
loc,지역명
cnt_hos,병원수
bed,격리병상수
remain_bed,남은 격리병상수
pop,총인구
nurse,요양기관수
total60,60세 이상 - 계
60-64,60~64세
65-69,65~69세
70-74,70~74세


In [ ]:
sido.columns = columns
sido.head()

,loc,cnt_hos,bed,remain_bed,pop,nurse,total60,60-64,65-69,70-74,75-79,80-84,85-,conf,death
0,강원도,18.0,539.0,75.0,1347375.0,34.0,369970,116558,81374,56864,53119,38211,23844,440985.0,746
1,경상남도,35.0,940.0,176.0,3295942.0,129.0,774696,253193,179622,128375,99667,70217,43622,1052389.0,1134
2,경상북도,29.0,1046.0,271.0,2583157.0,111.0,719154,213011,162533,119640,100383,75400,48187,731978.0,1203
3,광주광역시,19.0,240.0,55.0,1477573.0,60.0,290547,96662,65287,52134,38610,23826,14028,518968.0,520
4,대구광역시,18.0,876.0,249.0,2410700.0,75.0,559877,182705,133421,95965,73211,47435,27140,734292.0,1243


In [ ]:
# train data
X = sido.drop(columns=['loc','death'])
# target data
y = sido['death']

# train set : test set = 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
# train set : validation set = 8:2
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.2)

In [ ]:
print('train set', X_train.shape)
print('validation set', X_valid.shape)
print('test set', X_test.shape)

train set (28, 13)
validation set (8, 13)
test set (10, 13)


In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = lr.predict(X_test)

test set으로 예측한 사망자 수 y_pred

In [ ]:
plt.scatter(y_test, y_pred, alpha=0.4)
plt.xlabel("Actual death")
plt.ylabel("Predicted death")
plt.show()

사망자 수가 적을 때는 비교적 가깝게 맞추지만, 사망자 수가 많을 때는 오차가 있다.

In [ ]:
r2 = r2_score(y_test, y_pred)
print('r2 score is ' + str(r2))

r2 score is 0.9231901806048168


In [ ]:
regressionSummary(y_train, lr.predict(X_train))


Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 52.9292
            Mean Absolute Error (MAE) : 41.1545
          Mean Percentage Error (MPE) : -18.6960
Mean Absolute Percentage Error (MAPE) : 30.6972


In [ ]:
regressionSummary(y_valid, lr.predict(X_valid))


Regression statistics

                      Mean Error (ME) : -47.0780
       Root Mean Squared Error (RMSE) : 102.8208
            Mean Absolute Error (MAE) : 81.6056
          Mean Percentage Error (MPE) : -14.4696
Mean Absolute Percentage Error (MAPE) : 26.4044


학습 데이터셋이 적으므로, k-fold cross validation을 통해 정확도를 향상시켜보자

In [ ]:
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

lr = LinearRegression()
scores = cross_val_score(lr, X_train, y_train)
print(scores.mean())

0.5235680150410925


In [ ]:
scores_cv3 = cross_val_score(lr, X_train, y_train, cv=3)
print(scores_cv3.mean())

0.20105557246503528


In [ ]:
scores_cv7 = cross_val_score(lr, X_train, y_train, cv=7)
print(scores_cv7.mean())

0.7181552340210093


In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred, alpha=0.4)
plt.xlabel("Actual death")
plt.ylabel("Predicted death")
plt.show()

In [ ]:
r2 = r2_score(y_test, y_pred)
print('r2 score is ' + str(r2))

r2 score is 0.9135948842663688


교차 검증을 거친 후 성능이 좋아진 것을 알 수 있다.

In [ ]:
regressionSummary(y_train, lr.predict(X_train))


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 42.8794
            Mean Absolute Error (MAE) : 34.0965
          Mean Percentage Error (MPE) : -9.7362
Mean Absolute Percentage Error (MAPE) : 27.5061


# Lasso Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.278e+04, tolerance: 2.032e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Lasso(alpha=0.1)

In [ ]:
y_pred = clf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print('r2 score is ' + str(r2))

r2 score is 0.7111573214209229


# Ridge Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

clf = Ridge(alpha=0.3)
clf.fit(X_train, y_train)

Ridge(alpha=0.3)

In [ ]:
y_pred = clf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print('r2 score is ' + str(r2))

r2 score is 0.9111262483198533


# 실제 데이터에 모델 적용

In [ ]:
pred_sido = pd.read_csv("/content/drive/MyDrive/2022_IPS/final_data/시도명_시군구명(predict).csv").drop(columns=['Unnamed: 0'])
pred_sido.head()

,시도명,시군구명,병원수,격리병상수,남은 격리병상수,총인구,요양기관수,60세 이상 - 계,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상,확진자수
0,강원도,강릉시,3.0,79.0,24.0,216542.0,3.0,61339,18656,13586,9988,8900,6185,4024,65832.0
1,강원도,동해시,2.0,66.0,1.0,87801.0,3.0,24762,7662,5655,4087,3680,2278,1400,28349.0
2,강원도,삼척시,1.0,51.0,8.0,65939.0,1.0,21481,6050,4525,3591,3311,2377,1627,17543.0
3,강원도,속초시,2.0,119.0,5.0,80054.0,2.0,22194,7239,4986,3556,3055,2062,1296,27323.0
4,강원도,영월군,1.0,27.0,1.0,36282.0,1.0,14753,4248,3297,2309,2283,1671,945,9242.0


In [ ]:
columns = ['loc_sido','loc_gun','cnt_hos','bed','remain_bed','pop','nurse','total60','60-64','65-69','70-74','75-79','80-84','85-','conf']
pred_sido.columns = columns
pred_sido.head()

,loc_sido,loc_gun,cnt_hos,bed,remain_bed,pop,nurse,total60,60-64,65-69,70-74,75-79,80-84,85-,conf
0,강원도,강릉시,3.0,79.0,24.0,216542.0,3.0,61339,18656,13586,9988,8900,6185,4024,65832.0
1,강원도,동해시,2.0,66.0,1.0,87801.0,3.0,24762,7662,5655,4087,3680,2278,1400,28349.0
2,강원도,삼척시,1.0,51.0,8.0,65939.0,1.0,21481,6050,4525,3591,3311,2377,1627,17543.0
3,강원도,속초시,2.0,119.0,5.0,80054.0,2.0,22194,7239,4986,3556,3055,2062,1296,27323.0
4,강원도,영월군,1.0,27.0,1.0,36282.0,1.0,14753,4248,3297,2309,2283,1671,945,9242.0


In [ ]:
predictors = pred_sido.drop(columns=['loc_sido','loc_gun'])

In [ ]:
pred_death=[]
for i in range(0, 206):
    pred_death.append(clf.predict(predictors.loc[[i]]))

In [ ]:
pred_sido_death = pred_sido
pred_sido_death['pred_death'] = pd.DataFrame(pred_death)
pred_sido_death

,loc_sido,loc_gun,cnt_hos,bed,remain_bed,pop,nurse,total60,60-64,65-69,70-74,75-79,80-84,85-,conf,pred_death
0,강원도,강릉시,3.0,79.0,24.0,216542.0,3.0,61339,18656,13586,9988,8900,6185,4024,65832.0,95.136241
1,강원도,동해시,2.0,66.0,1.0,87801.0,3.0,24762,7662,5655,4087,3680,2278,1400,28349.0,-16.317735
2,강원도,삼척시,1.0,51.0,8.0,65939.0,1.0,21481,6050,4525,3591,3311,2377,1627,17543.0,26.547888
3,강원도,속초시,2.0,119.0,5.0,80054.0,2.0,22194,7239,4986,3556,3055,2062,1296,27323.0,-22.785828
4,강원도,영월군,1.0,27.0,1.0,36282.0,1.0,14753,4248,3297,2309,2283,1671,945,9242.0,14.742016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,충청북도,제천시,2.0,32.0,3.0,134768.0,6.0,39959,12590,9246,6191,5764,3910,2258,42422.0,60.405401
202,충청북도,증평군,0.0,0.0,0.0,37183.0,1.0,8734,2788,1954,1337,1225,887,543,15155.0,27.913467
203,충청북도,진천군,1.0,67.0,17.0,89514.0,1.0,18410,5699,4030,2787,2445,2112,1337,34557.0,21.001106
204,충청북도,청주시,7.0,445.0,73.0,855326.0,14.0,162294,56336,37220,25076,20488,14104,9070,297157.0,254.444241


In [ ]:
pred_sido_death.to_csv("릿지회귀_예측결과.csv", mode='w', encoding='utf-8-sig')